No lie this one kicked my butt.
I did look for a hint which ended up giving away what part 2 was going to be, which did help me reframe my thinking and come up with a solution.

I had known it would come down to finding the "minimum" chunks and with small enough parts we can cache and greatly speed up...and we did!

It's amazing how fast the correct part2 runs given how slow my initial part *1* was!

In [169]:
from functools import cache

In [170]:
num_paths = {
    '7': {
        '>': '8',
        'v': '4',
    },
    '8': {
        '>': '9',
        'v': '5',
        '<': '7',
    },
    '9': {
        '<': '8',
        'v': '6',
    },
    '4': {
        '^': '7',
        '>': '5',
        'v': '1',
    },
    '5': {
        '^': '8',
        '<': '4',
        '>': '6',
        'v': '2',
    },
    '6': {
        '^': '9',
        '<': '5',
        'v': '3',
    },
    '1': {
        '^': '4',
        '>': '2',
    },
    '2': {
        '^': '5',
        '<': '1',
        '>': '3',
        'v': '0',
    },
    '3': {
        '^': '6',
        '<': '2',
        'v': 'A',
    },
    '0': {
        '^': '2',
        '>': 'A',
    },
    'A': {
        '^': '3',
        '<': '0',
    },
}

dir_paths = {
    '^': {
        'v': 'v',
        '>': 'A'
    },
    'A': {
        '<': '^',
        'v': '>'
    },
    '<': {
        '>': 'v'
    },
    'v': {
        '^': '^',
        '<': '<',
        '>': '>'
    },
    '>': {
        '<': 'v',
        '^': 'A'
    }
}

In [171]:
from collections import deque
@cache 
def path(s, d, num_pad=True) -> tuple[str, ...]:
    if s == d:
        return ('',) 
    paths = num_paths if num_pad else dir_paths
    mn = None
    soln = []
    q = deque(paths[s].items())
    # print(q)
    while q:
        p, c = q.popleft()
        if mn is not None and len(p) > mn:
            break
        if c == d:
            if len(soln) == 0:
                soln.append(p)
                mn = len(p)
            else:
                if len(p) < mn:
                    soln = [p]
                    mn = len(p)
                elif len(p) == mn:
                    soln.append(p)
        else:
            for k,v in paths[c].items():
                q.append((p + k, v))
            q = deque(sorted(q, key=lambda t: len(t[0])))
    return tuple([s for s in soln])
            
            


In [172]:
@cache
def paths_for_code(code, num_pad=True) -> tuple[str,...]:
    src = 'A'
    que = ['']
    for c in code:
        paths = path(src, c, num_pad)
        que = [q+p for q in que for p in paths]
        src = c
        que = [q+'A' for q in que]
    min_len = min(len(q) for q in que)
    return tuple([q for q in que if len(q) == min_len])

In [173]:
pfc = paths_for_code('029A', True)
pfc

('<A^A^^>AvvvA', '<A^A^>^AvvvA', '<A^A>^^AvvvA')

In [174]:
pfc2 = paths_for_code(pfc[0], False)
# pfc2
min(len(s) for s in pfc2)

28

In [182]:
@cache
def slow_robots(code, n) -> int:
    if n == 0:
        return len(code)
    paths = paths_for_code(code, False)
    recur_paths = [slow_robots(p, n-1) for p in paths]
    return min(recur_paths)

In [183]:
@cache
def robots(code, n) -> int:
    if n == 0:
        return len(code)
    # split on "A" but keep the "A" (drop last because it's extra)
    # ex: turns "^^AvvA" into ["^^A", "vvA"]
    splits = [s + 'A' for s in code.split('A')][:-1]
    # by breaking this into tiny chunks we can fully leverage @cache
    ans = 0
    for s in splits:
        paths = paths_for_code(s, False)
        ans += min(robots(p, n-1) for p in paths)
    return ans

In [184]:
import math
def part1(code) -> int:
    round1 = paths_for_code(code, True)
    mn = math.inf
    for r in round1:
        mn = min(mn, robots(r, 2))
    return mn * int(code[:-1])

In [185]:
sum([part1(s) for s in """029A
980A
179A
456A
379A""".splitlines()])

126384

In [186]:
sum([part1(s) for s in """149A
582A
540A
246A
805A""".splitlines()])

164960

In [187]:
import math
def part2(code) -> int:
    round1 = paths_for_code(code, True)
    mn = math.inf
    for r in round1:
        mn = min(mn, robots(r, 25))
    return mn * int(code[:-1])

In [188]:
sum([part2(s) for s in """149A
582A
540A
246A
805A""".splitlines()])

205620604017764